In [22]:
import requests
import urllib
from bs4 import BeautifulSoup
import bs4
import pandas as pd

In [2]:
r = requests.get(url="https://www.daft.ie/price-register/dublin-city/", params={"min_price":25000,
                                                                            "max_price":1000000,"pagenum":3677},
                timeout=100, proxies=urllib.request.getproxies(),
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})

In [3]:
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
ppr = soup.find_all("div", {"class":"priceregister-searchresult"})

In [8]:
sr = ppr[0]
sr

<div class="priceregister-searchresult">
<span class="priceregister-address">
        29
        <a href="/price-register/dublin/blackrock/carysfort-woods/">Carysfort Woods</a>,
        <a href="/price-register/dublin/blackrock/">Blackrock</a>,
        <a href="/price-register/dublin/">Co. Dublin</a>
</span>
<span class="priceregister-dwelling-details">
<b>€547,500</b>
        | 04/01/10
        | Second-Hand Dwelling House/Apartment
            </span>
<span class="priceregister-status-box">
<img alt="SOLD" height="15" src="https://c1.dmstatic.com/890/i/sold_icon.png" width="38"/>
</span>
</div>

In [12]:
addr = sr.find("span", "priceregister-address").contents
addr

['\n        29\n        ',
 <a href="/price-register/dublin/blackrock/carysfort-woods/">Carysfort Woods</a>,
 ',\n        ',
 <a href="/price-register/dublin/blackrock/">Blackrock</a>,
 ',\n        ',
 <a href="/price-register/dublin/">Co. Dublin</a>,
 '\n']

In [16]:
[type(x) for x in addr]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [26]:
addr[0].strip()

'29'

In [32]:
" ".join([x.contents[0].strip() if type(x) is bs4.element.Tag else x.strip() for x in addr]).replace(" ,", ",").strip()

'29 Carysfort Woods, Blackrock, Co. Dublin'

In [34]:
det = sr.find("span", "priceregister-dwelling-details").contents
det

['\n',
 <b>€547,500</b>,
 '\n        | 04/01/10\n        | Second-Hand Dwelling House/Apartment\n            ']

In [39]:
price = det[1].contents[0].strip()
price

'€547,500'

In [40]:
info = det[2].strip().split("|")
info = [x.strip() for x in info]
info

['', '04/01/10', 'Second-Hand Dwelling House/Apartment']

In [26]:
r = requests.get(url="https://www.daft.ie/price-register/dublin-city/", params={"min_price":25000,
                                                                            "max_price":1000000,"pagenum":4677},
                timeout=100, proxies=urllib.request.getproxies(),
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})

In [27]:
soup = BeautifulSoup(r.content, 'html.parser')

In [28]:
ppr = soup.find_all("div", {"class":"priceregister-searchresult"})

In [29]:
ppr

[]

In [59]:
def get_page_results(page):
    r = requests.get(url="https://www.daft.ie/price-register/dublin-city/", params={"min_price":25000,
                                                                            "max_price":1000000,"pagenum":page},
                timeout=100, proxies=urllib.request.getproxies(),
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})
    soup = BeautifulSoup(r.content, 'html.parser')
    ppr = soup.find_all("div", {"class":"priceregister-searchresult"})
    ret = []
    for sr in ppr:
        addr = sr.find("span", "priceregister-address").contents
        ret_addr = " ".join(["".join(x.contents).strip()
                                       if type(x) is bs4.element.Tag
                                       else x.strip()
                                   for x in addr])\
                             .replace(" ,", ",").strip()
        det = sr.find("span", "priceregister-dwelling-details").contents
        ret_price = det[1].contents[0].strip()
        info = det[2].strip().split("|")
        ret_detail = [x.strip() for x in info]
        ret.append((ret_addr, ret_price, ret_detail))
        
    return ret

In [54]:
all_pages = [get_page_results(x) for x in range(100)]

In [55]:
pd.DataFrame([x for sl in all_pages for x in sl], columns=["address", "price", "details"]).to_csv("../data/daft.csv", sep="\t", index=False)

In [60]:
all_pages = [get_page_results(x) for x in range(4000)]

In [61]:
pd.DataFrame([x for sl in all_pages for x in sl], columns=["address", "price", "details"]).to_csv("../data/daft.csv", sep="\t", index=False)